# Synapse demo
Welcome to the synapse demo! I don't have many analytical skills so this will focus on using the synapse client to download data and store results.

## Get the files
You first have to install the `synapseclient` via pip. This will install a `.synapseConfig` file in your home directory that you must then edit to include your user name and password. Then you can use the `login` command shown below.

In [1]:
import synapseclient
import pandas

## note: i have added my login information to the .synapseClient'
## file
syn=synapseclient.login()

Welcome, Sara Gosline!



Every file, table and project in Synapse has a unique identifier that allows us to call the `get` command. They represent entities, of which files are one type. So we can get the file entity, extract the path, and read the file.

In [2]:
fpath=syn.get('syn21817531').path
print(fpath)
tab = pandas.read_table(fpath,sep='\t',comment='!')

c:/users/gosl241/.synapsecache/690/51216690/gse30589_series_matrix.txt


Next we do a bit of munging, and a little bit of visualization. (I can't do more because the tool are in R).

In [3]:
#lets clean up the identifiers
tab=tab.rename(index=tab['ID_REF'])
tab=tab.drop("ID_REF",1)

#first we compute some summary statistics
probeVar=tab.var(axis=1)
probeMean=tab.mean(axis=1)
probeMin=tab.min(axis=1)
probeMax=tab.max(axis=1)

#sometimes i like to visualize the most variable probes
mostVar=list(probeVar.sort_values(ascending=False)[0:20].index)
tab.loc[mostVar,:].style.background_gradient(cmap='viridis')


,GSM758827,GSM758828,GSM758829,GSM758830,GSM758831,GSM758832,GSM758833,GSM758834,GSM758835,GSM758836,GSM758837,GSM758838,GSM758839,GSM758840,GSM758841,GSM758842,GSM758843,GSM758844,GSM758845,GSM758846,GSM758847,GSM758848,GSM758849,GSM758850,GSM758851,GSM758852,GSM758853,GSM758854,GSM758855,GSM758856,GSM758857,GSM758858,GSM758859
225681_at,1.780000,2.040000,1.820000,1.670000,2.350000,2.210000,1.940000,1.870000,2.150000,3.330000,3.740000,3.380000,3.360000,3.250000,3.340000,3.240000,3.250000,3.360000,11.420000,11.430000,11.330000,11.550000,11.370000,11.460000,11.140000,11.180000,11.160000,12.260000,12.160000,12.090000,12.170000,12.290000,12.240000
1554981_at,1.980000,2.070000,1.900000,2.050000,1.720000,1.660000,1.860000,1.900000,1.840000,3.080000,3.210000,2.900000,2.900000,2.900000,2.920000,2.760000,2.790000,2.790000,11.430000,11.590000,11.390000,11.490000,11.430000,11.360000,11.460000,11.590000,11.560000,11.090000,11.080000,11.150000,11.360000,11.110000,11.000000
201163_s_at,1.630000,1.610000,1.710000,1.450000,1.640000,1.750000,1.750000,1.660000,1.630000,2.700000,2.700000,2.700000,2.700000,2.770000,2.780000,2.590000,2.650000,2.550000,9.780000,9.710000,9.530000,9.770000,9.170000,9.280000,9.750000,9.850000,9.770000,10.530000,10.120000,10.200000,10.560000,10.430000,10.510000
1570561_at,1.740000,1.850000,1.890000,1.760000,1.770000,1.700000,1.750000,1.840000,1.980000,3.250000,3.690000,3.090000,3.100000,2.940000,2.970000,3.020000,2.970000,3.020000,9.790000,9.280000,9.660000,9.510000,8.940000,9.330000,8.100000,8.140000,8.160000,11.810000,11.990000,11.820000,10.860000,10.450000,10.930000
AFFX-HUMRGE/M10098_5_at,3.370000,4.260000,4.010000,4.210000,4.240000,4.920000,2.640000,2.770000,2.850000,10.660000,10.700000,10.610000,10.640000,10.360000,10.740000,7.370000,7.300000,7.740000,9.410000,8.160000,8.780000,8.960000,8.730000,9.000000,7.590000,7.790000,7.730000,14.940000,14.890000,14.860000,14.330000,13.600000,14.330000
203914_x_at,2.770000,2.760000,2.920000,2.660000,2.990000,2.790000,2.980000,2.910000,3.340000,3.700000,3.640000,3.310000,3.390000,3.720000,3.470000,4.010000,4.110000,4.260000,9.420000,9.450000,9.210000,9.390000,9.260000,9.330000,9.300000,9.430000,9.360000,11.730000,11.470000,11.500000,11.750000,11.620000,11.730000
225728_at,2.040000,1.760000,2.350000,2.010000,2.010000,2.010000,2.160000,2.040000,2.150000,3.990000,3.560000,3.410000,3.690000,3.210000,3.460000,2.860000,3.270000,3.160000,9.460000,9.520000,9.390000,9.490000,9.270000,9.250000,9.410000,9.390000,9.340000,10.290000,10.200000,10.270000,9.990000,10.050000,10.080000
212097_at,5.570000,5.350000,5.140000,5.430000,5.740000,5.620000,6.750000,6.700000,6.300000,4.280000,4.030000,4.400000,4.480000,4.270000,4.570000,6.580000,7.390000,7.200000,12.330000,12.330000,12.200000,12.360000,12.170000,12.230000,12.180000,12.270000,12.160000,12.300000,12.270000,12.350000,12.410000,12.400000,12.480000
201162_at,5.030000,5.330000,5.190000,4.700000,4.400000,4.820000,4.250000,4.480000,4.450000,6.250000,6.420000,6.320000,6.270000,6.080000,6.370000,6.070000,5.860000,5.620000,11.480000,11.530000,11.510000,11.480000,11.530000,11.510000,11.670000,11.630000,11.630000,12.400000,12.420000,12.500000,12.210000,12.320000,12.220000
209031_at,3.450000,2.940000,3.270000,3.430000,2.880000,3.250000,3.340000,3.510000,3.280000,4.910000,4.800000,5.150000,4.770000,5.400000,4.710000,4.770000,5.040000,4.910000,10.270000,10.240000,10.270000,10.100000,10.050000,10.140000,10.030000,9.880000,9.750000,11.290000,11.160000,11.240000,10.850000,10.960000,10.930000


## Create new analysis output
Next we might want to store the results, so that someone else can process it. There are a few ways to do this. 

In [4]:
summary=pandas.DataFrame({'variance':probeVar,'mean':probeMean,'min':probeMin,'max':probeMax})

summary.to_csv('arraySummaryStats.csv')

The first is to write to a file and store the `File` entity like the one we got. 

In [5]:
summary.to_csv('arraySummaryStats.csv')
ent=syn.store(synapseclient.File('arraySummaryStats.csv',parent='syn21814566'))


##################################################
 Uploading file to Synapse storage 
##################################################



The second way is to store as a table. For this it shows up on the [tables tab](https://www.synapse.org/#!Synapse:syn21788423/tables/) of our project.

In [6]:
summary['probes']=summary.index[0:,]
table = synapseclient.table.build_table('Array Summary Statistics', 'syn21788423', summary)
table = syn.store(table)